In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from utils import estimate_bias, estimate_bias_coefs,norm_quantile

In [2]:
vol = jlc.load_tifvol("./YCsgA_Exp2_Pos1.tif")
vol = vol[:,:,:vol.shape[2]//2].astype(float)
vol = norm_quantile(vol,alpha=0.001,clip=True)
coefs = estimate_bias_coefs(vol)
vol *= coefs.reshape(-1,1,1)
bias = estimate_bias(vol)
vol -= bias[None]
vol = norm_quantile(vol,alpha=0.001,clip=True)

In [3]:
jlc.inspect_vol(vol)

In [163]:
frame1 = vol[100]
frame2 = vol[300]
plt.figure(figsize=(10,5))
plt.subplot(131)
plt.imshow(frame1, cmap='gray')
plt.subplot(132)
plt.imshow(frame2, cmap='gray')
plt.subplot(133)
plt.imshow(np.concatenate((frame1[:,:,None],
                           frame2[:,:,None], 
                           frame2[:,:,None]*0), axis=2), cmap='gray')

In [167]:

#translate_search = np.linspace(-5,5,51)
translate_search = np.arange(-5,5+1)
use_sq_err = True
max_corr = -float('inf')
best_translation = (0, 0)
corr_mat = np.zeros((len(translate_search), len(translate_search)))
for x in translate_search:
    for y in translate_search:
        translation = np.float32([[1,0,x],[0,1,y]])
        frame2_translated = cv2.warpAffine(frame2, translation, (frame2.shape[1], frame2.shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)
        if use_sq_err:
            corr = -np.mean((frame1-frame2_translated)**2)
        else:
            corr = np.corrcoef(frame1.flatten(), frame2_translated.flatten())[0,1]
        corr_mat[translate_search==x, translate_search==y] = corr
        if corr > max_corr:
            max_corr = corr
            best_translation = (x, y)

print("Best translation:", best_translation)


Best translation: (-2, 4)


In [165]:
corr_mat_g = scipy.ndimage.gaussian_filter(corr_mat,sigma=3)
#find max idx
max_idx = np.unravel_index(np.argmax(corr_mat_g), corr_mat_g.shape)
print("Max idx:", max_idx)
print("Best x:", translate_search[max_idx[0]])
print("Best y:", translate_search[max_idx[1]])

Max idx: (16, 44)
Best x: -1.7999999999999998
Best y: 3.8000000000000007


In [ ]:
corr_mat_big = cv2.resize(corr_mat, (0,0), fx=10, fy=10, interpolation=cv2.INTER_LANCZOS4)
d = translate_search[1] - translate_search[0]
translate_search_big = np.linspace(translate_search[0] -d/2,
                                   translate_search[-1]+d/2,corr_mat_big.shape[0])
#rainbow colors
plt.imshow(corr_mat_big, cmap='jet')
_=plt.xticks(np.arange(0,len(translate_search_big)), np.round(translate_search_big, 2))
_=plt.yticks(np.arange(0,len(translate_search_big)), np.round(translate_search_big, 2))

In [186]:
max_idx = np.unravel_index(np.argmax(corr_mat_big), corr_mat_big.shape)
print("Max idx:", max_idx)
print("Best x:", translate_search_big[max_idx[0]])
print("Best y:", translate_search_big[max_idx[1]])

Max idx: (37, 91)
Best x: -1.7660550458715596
Best y: 3.68348623853211


In [190]:
translation = np.float32([[1,0,translate_search_big[max_idx[0]]],
                          [0,1,translate_search_big[max_idx[1]]]])
frame2_translated = cv2.warpAffine(frame2, translation, (frame2.shape[1], frame2.shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)
plt.figure(figsize=(10,5))
plt.subplot(131)
plt.imshow(frame1, cmap='gray')
plt.subplot(132)
plt.imshow(frame2_translated, cmap='gray')
plt.subplot(133)
plt.imshow(np.concatenate((frame1[:,:,None],
                           frame2_translated[:,:,None], 
                           frame2_translated[:,:,None]*0), axis=2), cmap='gray')

In [ ]:
corr_mat_g = scipy.ndimage.gaussian_filter(corr_mat,sigma=3)
#find max idx
max_idx = np.unravel_index(np.argmax(corr_mat_g), corr_mat_g.shape)
print("Max idx:", max_idx)
print("Best x:", translate_search[max_idx[0]])
print("Best y:", translate_search[max_idx[1]])

Max idx: (16, 44)
Best x: -1.7999999999999998
Best y: 3.8000000000000007


In [168]:
plt.imshow(corr_mat)
plt.colorbar()
_=plt.xticks(np.arange(0,len(translate_search)), np.round(translate_search, 2))
_=plt.yticks(np.arange(0,len(translate_search)), np.round(translate_search, 2))

In [44]:
plt.imshow(scipy.ndimage.gaussian_filter(vol[:,:,300], sigma=(0,3)))

In [153]:
def soft_threshold(start=0,stop=1,power=1): 
    to_interval = lambda x: np.clip((x-start)/(stop-start),0,1)
    return lambda x: h(to_interval(x),power)
def h(x,p):
    return (x<0.5)*0.5*2**p*x**p+(x>=0.5)*(1-0.5*2**p*(1-x)**p)

t = np.linspace(-1,2,300)
for p in [0.5,1,2,3]:
    f = soft_threshold(0,1,p)
    plt.plot(t,f(t))

In [155]:
jlc.inspect_vol(soft_threshold(0.1,0.3,2)(vol))

In [5]:
translate_search = np.arange(-8,8+1)
slice_x = slice(None)
slice_y = slice(None)
sigma = 0
upscale = 10
frame1 = vol[int(vol.shape[0]*0.5):int(vol.shape[0]*0.5)+1].mean(0)[slice_x,slice_y]
d = translate_search[1] - translate_search[0]
translate_search_big = np.linspace(translate_search[0] -d/2,
                                   translate_search[-1]+d/2,upscale*len(translate_search))
corr_mat = np.zeros((len(translate_search), len(translate_search)))
if sigma>0:
    frame1 = scipy.ndimage.gaussian_filter(frame1, sigma)
best_translation = []
for f_i in range(vol.shape[0]):
    max_corr = -float('inf')
    for x in translate_search:
        for y in translate_search:  
            frame2 = vol[f_i, slice_x, slice_y]
            translation = np.float32([[1,0,x],[0,1,y]])
            frame2_translated = cv2.warpAffine(frame2, translation, (frame2.shape[1], frame2.shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)
            if sigma>0:
                frame2_translated_g = scipy.ndimage.gaussian_filter(frame2_translated, sigma)
            else:
                frame2_translated_g = frame2_translated
            corr = -np.mean((frame1-frame2_translated_g)**2)
            corr_mat[translate_search==x, translate_search==y] = corr
    corr_mat_big = cv2.resize(corr_mat, (0,0), fx=upscale, fy=upscale, interpolation=cv2.INTER_LANCZOS4)        
    max_idx = np.unravel_index(np.argmax(corr_mat_big), corr_mat_big.shape)
    best_translation.append((translate_search_big[max_idx[0]],translate_search_big[max_idx[1]]))
    if f_i%50==0:
        print(f"done with frame {f_i}/{vol.shape[0]-1}")


done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414


0.12203762247255061

In [22]:
jlc.inspect_vol((vol+bias)/(bias+bias.mean()/3),vmin=0,vmax=3)

MemoryError: Unable to allocate 927. MiB for an array with shape (415, 684, 428) and data type float64

In [6]:
translated_vol = np.zeros_like(vol)
for f_i in range(len(vol)):
    translation = np.float32([[1,0,best_translation[f_i][0]],[0,1,best_translation[f_i][1]]])
    translated_vol[f_i] = cv2.warpAffine(vol[f_i], translation, (vol[f_i].shape[1], vol[f_i].shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)

In [7]:
jlc.inspect_vol(translated_vol.transpose(0,2,1))

In [205]:
jlc.inspect_vol(np.concatenate((vol, translated_vol), axis=2))

In [31]:
plt.plot(np.array(best_translation)[:,0], label='x')
plt.plot(np.array(best_translation)[:,1], label='y')

In [100]:
from scipy.ndimage import gaussian_filter
smooth_translation = np.array(best_translation)
smooth_translation = gaussian_filter(smooth_translation, sigma=[3,0])
plt.plot(smooth_translation[:,0], label='x')
plt.plot(smooth_translation[:,1], label='y')

In [68]:
plt.plot(np.array(best_translation)[:,0],np.array(best_translation)[:,1])

In [131]:
plt.imshow(vol[:,:,0])

In [132]:
a = gaussian_filter(vol[200,:,0],2)
b = gaussian_filter(vol[205,:,0],2)
plt.scatter(a,b)
plt.plot([0,1],[0,1],color="red")
plt.xlim(min(a.min(),b.min()), max(a.max(),b.max()))
plt.ylim(min(a.min(),b.min()), max(a.max(),b.max()))



(0.021767714915564087, 0.3470474157935479)

In [127]:
b.shape

(684,)

In [139]:
plt.plot(a)
plt.plot(b)

In [128]:
cc = []
for t in translate_search:
    b_translated = cv2.warpAffine(b, np.float32([[1,0,t],[0,1,0]]), (1,b.shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)[:,0]
    corr = np.corrcoef(a,b_translated)[0,1]
    cc.append(corr)
plt.plot(translate_search, cc)